# Propagation effects

In this notebook we will have a look at the individuel propagation effects that can be included.

## Setup

In [ ]:
import numpy as np
from acoustics.signal import OctaveBand
from acoustics import Signal
from auraliser import Auraliser, mono
from auraliser.generator import Sine, Noise
#import seaborn as sns
from geometry import Point
from IPython.display import Audio
import matplotlib
matplotlib.rcParams['figure.figsize'] = (9.0, 6.0)
%matplotlib inline
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
from streaming import Stream
from streaming.signal import constant

In [ ]:
fs = 44100.0      # Sample frequency
duration = 20.      # Duration in seconds
df = 50.0           # Frequency resolution (impedances reflections)
nsamples = int(fs*duration)

dt = 1.0/fs                             # Seconds per sample
t = np.arange(0.0, duration, dt)        # Time vector

frequency = 1000.0

model = Auraliser(duration=duration)
model.settings['fs'] = fs

We will need a source

In [ ]:
speed = 100.0
x = np.ones_like(t) * speed * (t - duration/2.0)    # Source moves along the x-axis.
y = np.ones_like(t) * 0.01
z = np.ones_like(t) * 200.0   # Altitude of source
src = model.add_source(name='source', position=np.vstack((x,y,z)).T)
#src = model.add_source(name='source', position=Point(0.0,0.0,0.0))
subsrc = src.add_subsource(name='subsource')

emitting a sine wave and pink noise

In [ ]:
sine = subsrc.add_virtualsource('sine', signal = Sine(frequency=frequency), level=145.)
noise = subsrc.add_virtualsource('pink', signal = Noise(color='pink'), level=140.)

and a receiver.

In [ ]:
rcv = model.add_receiver(name='receiver', position=Point(0.0,0.0,4.0))

Because we would like to consider the propagation effects individually we turn them all of for now.

In [ ]:
model.settings['spreading']['include'] = False
model.settings['doppler']['include'] = False
model.settings['atmospheric_absorption']['include'] = False
model.settings['reflections']['include'] = False
model.settings['turbulence']['include'] = False

model.settings['doppler']['purge_zeros'] = False

In [ ]:
ylim = (0.0, 4000.0)

In [ ]:
model.settings

## Effects

Now that we have a basic model to work with we look at the propagation effects one by one.

In [ ]:
signal = Signal(mono( rcv.auralise() ).toarray(), fs)

In [ ]:
signal

In [ ]:
Audio(data=signal, rate=fs)

### Distance reduction

In the far field sound spreads out spherically. This results in a decrease in amplitude with increase in distance.

In [ ]:
model.settings['spreading']['include'] = True
signal = Signal(mono( rcv.auralise() ).toarray(), fs)
model.settings['spreading']['include'] = False

In [ ]:
signal.shape

In [ ]:
Audio(data=signal, rate=fs)

In [ ]:
_ = signal.plot_levels()

### Time delay

Sound has a limited velocity resulting in a delay between emission and receivement. Movement of source and/or receiver can result in a variable time delay.

Besides the obvious time delay this also results in a Doppler shift in frequency.

In [ ]:
model.settings['doppler']['include'] = True
signal = Signal(mono( rcv.auralise() ).toarray(), fs)
model.settings['doppler']['include'] = False

In [ ]:
Audio(data=signal, rate=fs)

In [ ]:
_ = signal.plot_spectrogram(ylim=ylim)

### Atmospheric absorption

Relaxation effects in the atmosphere result in additional attenuatation. 

In [ ]:
model.settings['atmospheric_absorption']['include'] = True
signal = Signal(mono( rcv.auralise() ).toarray(), fs)
model.settings['atmospheric_absorption']['include'] = False

In [ ]:
Audio(data=signal, rate=fs)

In [ ]:
_ = signal.plot_spectrogram(ylim=ylim)

In [ ]:
_ = signal.plot_levels()

### Atmospheric turbulence

In [ ]:
model.settings['turbulence']['include'] = True
signal = Signal(mono( rcv.auralise() ).toarray(), fs)
model.settings['turbulence']['include'] = False

In [ ]:
Audio(data=signal, rate=fs)

In [ ]:
_ = signal.plot_spectrogram(ylim=ylim)

In [ ]:
_ = signal.plot_levels()

In [ ]:
signal.leq()

In [ ]:
_ = signal.instantaneous_frequency().pick(3.0, 3.5).plot(ylim=(950, 1050))

### Ground reflection

A ground reflection without a propagation delay.

In [ ]:
from ism import Wall
frequencies = np.arange(0.0, fs/2.0, df)
impedance = np.ones_like(frequencies) + 1j*np.ones_like(frequencies)

groundcorners1 = [Point(-100.0, -100.0, 0.0), 
                  Point(100.0, -100.0, 0.0), 
                  Point(100.0, 100.0, 0.0), 
                  Point(-100.0, 100.0, 0.0) ]
ground1 = Wall(groundcorners1, Point(0.0, 0.0, 0.0), impedance)

model.geometry.walls = [ground1]

In [ ]:
model.settings['reflections']['include'] = True
model.settings['reflections']['force_hard'] = True

signal = Signal(mono( rcv.auralise() ).toarray(), fs)

model.settings['reflections']['include'] = False
model.settings['reflections']['force_hard'] = False

In [ ]:
Audio(data=signal, rate=fs)

In [ ]:
_ = signal.plot_spectrogram(ylim=ylim)

Because the propagation delay is not included, we do not get any interference.

In [ ]:
_ = signal.plot_levels()

### All effects

Finally, we enable all propagation effects.

In [ ]:
model.settings['spreading']['include'] = True
model.settings['doppler']['include'] = True
model.settings['atmospheric_absorption']['include'] = True
model.settings['reflections']['include'] = True
model.settings['reflections']['force_hard'] = True
model.settings['turbulence']['include'] = True


signal = Signal(mono( rcv.auralise() ).toarray(), fs)

model.settings['spreading']['include'] = False
model.settings['doppler']['include'] = False
model.settings['atmospheric_absorption']['include'] = False
model.settings['reflections']['include'] = False
model.settings['reflections']['force_hard'] = False
model.settings['turbulence']['include'] = False

In [ ]:
signal.samples

In [ ]:
Audio(data=signal, rate=fs)

In [ ]:
_ = signal.plot_spectrogram(ylim=ylim, clim=(-20, +70))

In [ ]:
_ = signal.plot_levels()